In [2]:
import json
from time import sleep
import datetime
from datetime import *
import shutil

try:
    from mal import config
    from mal import Anime
    from mal import AnimeSearch
    config.TIMEOUT = 20  # Import level config
except:
    print("MAL-Api nicht gefunden. Installieren mit 'pip install -U mal-api'")

In [2]:
def getAnime(malid):
    maldata = None
    timeout = 1
    while maldata is None:
        try:
            maldata = Anime(malid, timeout=10)
        except Exception as e:
            print(f"Fehler: {e}  Versuche nochmal in {int(timeout)} sec..")
            sleep(timeout)
            timeout=timeout*1.5
    return maldata

In [3]:
def loadjson():
    try:
        with open('archive.json', encoding="utf-8") as f:
            data = json.loads(f.read())
    except:
        with open('archive.json', encoding="utf-8") as f:
            data = json.loads(f.read()[f.read().index("["):])
    try:
        with open('dates.json', encoding="utf-8") as f:
            dates = json.loads(f.read())
    except:
        with open('dates.json', encoding="utf-8") as f:
            dates = json.loads(f.read()[f.read().index("["):])
    if len(data)*len(dates) != 0:
        print("okay. Daten geladen")
        return data, dates

In [4]:
def list_cleanup(data, mal=False):
    clean = []
    clean_w = [] #watchlist at the end or beginning
    for i in range(len(data)):
        old = data[i]
        try:
            if mal: 
                a=1/0 #go to except
            if min([len(str(s)) for s in list(old.values())])==0:
                a=1/0
            tmp = {"index":old["index"], 
                   "malid":old["malid"],
                   "title":old["title"],
                   "titen":old["titen"],
                   "titjp":old["titjp"],
                   "theme":old["theme"],
                   "imurl":old["imurl"],
                   "semes":old["semes"],
                   "progr":old["progr"],
                   "vdate":old["vdate"],
                   "statu":old["statu"]
                   }
        except Exception as es:
            if not mal: 
                print(f"Ungültiger Eintrag (id:{old['malid']}) entdeckt, lade neu von MAL.. ({es})")
            else:
                print("Lade von MAL..")
            
            maldata = getAnime(old["malid"])
                    
            tmp = {"index":old["index"], 
                   "malid":old["malid"],
                   "title":maldata.title,
                   "titen":maldata.title_english,
                   "titjp":maldata.title_japanese,
                   "theme":", ".join(maldata.themes),
                   "imurl":maldata.image_url.replace(".jpg","l.jpg").replace("ll.jpg","l.jpg"), #get large picture
                   "semes":old["semes"],
                   "progr":("00"+old["progr"][:2].replace("/",""))[-2:]+"/"+("00"+str(maldata.episodes).replace("None","??"))[-2:],
                   "vdate":old["vdate"],
                   "statu":old["statu"]
                   }
            #leere Einträge verhindern
            if tmp["titen"] in ["null", None, False, ""]:
                tmp["titen"] = tmp["title"]
            if tmp["theme"] in ["null", None, False, ""]:
                tmp["theme"] = " "
        print("")
        print(f"Eintrag {tmp['index']}:")
        print(tmp['title'])
        print(tmp["titen"])
        print(tmp["titjp"])
        print(tmp["theme"])
        print(tmp["statu"], tmp["progr"])
        print("")
        print("")
        if data[i]["statu"]!="W": 
            clean.append(tmp)
        else:
            clean_w.append(tmp)
    return clean+clean_w

In [5]:
def list_statusupdate(data, flag_needsort=False):

    print("Update-Datum eingeben mit Format: 31.01.2022 >> '3101[2022]' (Punkte und Jahr optional)")
    date = input()

    if len(date)>=4 and len(date)<=6:
        date += str(datetime.now().year)
    if len(date)==8:
        date = date[:2]+"."+date[2:4]+"."+date[4:]

    print("Datum: "+date)
    print("")
    print("STATUS UPDATEN: (Eingabe optional)")
    print("")

    for i in range(len(data)):
        if data[i]["statu"]=="W":
            print("Titel:", data[i]["title"])
            print("Folge:", data[i]["progr"])
            inc = input("increment:  +")
            sta = input("status change (W/F/D):")
            print("")
            if not inc in [None,"null","","0",0,"-0"]:
                try:
                    data[i]["progr"]=("00"+str(int(data[i]["progr"][:2])+int(inc)))[-2:]+data[i]["progr"][2:]
                    data[i]["vdate"]=date
                except:
                    print(f"ERROR: '{data[i]['progr']}' + '{inc}' kein gültiges increment")
                    print("")
            if sta!="":
                if sta in ["W","F","D"]:
                    data[i]["statu"] = sta
                    data[i]["vdate"]=date
                    flag_needsort = True
                else:
                    print(f"ERROR: '{sta}' kein gültiger Status")
                    print("")
            print("")

    print("Programm:") 

    ids = []
    for i in range(len(data)):
        if data[i]["statu"]=="W":
            ids.append(i)

    for i,j in zip(ids,range(len(ids))):
         print(j, "--", data[i]["title"])
    print("Neue Reihenfolge eingeben mit Format '0123': ")
    rf = input()
    if rf != "": #falls neue reihenfolge
        tmp = []
        if sorted(rf) == [str(k) for k in range(len(ids))]:
            for i in rf:
                tmp.append(data[ids[int(i)]])
            for i in range(len(ids)):
                data[sorted(ids)[i]] = tmp[i]
        else:
            print("")
            print("ERROR: Reihenfolge ist ungültig!")

    print("")
    print("FERTIG")
    if flag_needsort:
        print("Sortieren notwendig...")
        sleep(3)
        data = list_cleanup(data, mal = False)
    return data

In [6]:
def list_hinzufuegen(data):
    print("NEUEN ANIME HINZUFÜGEN/ALTEN WIEDERBELEBEN:")
    malid = input("MAL-ID (oder Titel für Suche):")
    maldata = None
    try:
        maldata = Anime(malid)
    except:
        print("")
        print("ID ungültig. Suchergebnisse:")
        try:
            search = AnimeSearch(malid)
            for i in range(min(30,len(search.results))):
                print(f'{(" "+str(i+1))[-2:]} -- {search.results[i].title} ({search.results[i].type})')
            i = input("Position: ")
            malid = search.results[int(i)-1].mal_id
            maldata = Anime(malid, timeout=10)
        except:
            print("")
            print("Suche nicht erfolgreich.")

    if maldata != None:
        print("")
        flag = False
        for i in range(len(data)):
            if data[i]["malid"] == malid:
                flag = True
                index = i
        if flag:
            print(f"Anime schon vorhanden. '{maldata.title}' wird wieder aktiviert.")
            data[i]["statu"] = "W"
        else:
            print(f"ID okay. '{maldata.title}' wird hinzugefügt")
            #semester
            if datetime.now().month in [4,5,6,7,8,9]:
                ts = str(datetime.now().year)[-2:]+"S"
            else:
                if datetime.now().month in [10,11,12]:
                    ts = str(datetime.now().year)[-2:]+"W"
                else:
                    ts = str(datetime.now().year-1)[-2:]+"W"
            #neuer eintrag
            tmp = {"index":str(len(data)+1), 
                   "malid":str(malid),
                   "semes":ts,
                   "vdate":"null",
                   "statu":"W",
                   "progr":"00/??"
                   }            
            data.append(tmp)
        print("")
        print("Bereinigung notwendig...")
        sleep(5)
        data = list_cleanup(data, mal=False)
        print("")
        print("Statusupdate notwendig...")
        sleep(5)
        data = list_statusupdate(data, flag_needsort=True)
    return data

In [7]:
def dates_sort(dates):
    for i in range(len(dates)):
        for j in range(i+1,len(dates)):
            if((datetime.fromisoformat(dates[i]["date"]).timestamp()-datetime.fromisoformat(dates[j]["date"]).timestamp())/(60*60*24) > 0):
                tmp = dates[i].copy()
                dates[i] = dates[j].copy()
                dates[j] = tmp
    return dates

In [8]:
def dates_next(dates):
    dates = dates_sort(dates)
    i = len(dates)
    while True:
        i+=-1
        if dates[i]["type"]!="S":
            break

    d = datetime.fromisoformat(dates[i]["date"])

    for j in range(1,5):
        tmp_d = d + timedelta(days=7*j)
        print("Datum für Sitzung:", str(tmp_d)[:16])
        ny = input("bestätigen? y/n? ")
        if ny in ["y","Y","j","J","1"]:
            tmp =  dates[i].copy()
            tmp["date"] = tmp_d.isoformat().replace(":00+","+")
            print("okay. Hinzugefügt:", tmp["title"], f"({tmp['type']}) @", str(tmp_d)[:16])
            dates.append(tmp)
        print("")  
    print("") 
    return dates

In [9]:
def dates_new(dates):
    print("(Zeitformat:", dates[-1]["date"].replace("+01:00",")"))
    d = input("Termin eingeben: ")
    d = d + "+01:00"
    try:
        datetime.fromisoformat(d)

        tmp = dates[-1].copy()
        tmp["date"] = d

        tmp["title"] = input("Titel: ")
        tmp["type"] = input("Typ Normal (A/B) oder Event (S): ")
        if not tmp["type"] in ["A","B","S"]:
            a=1/0
        dates.append(tmp)
        print("okay. Hinzugefügt:", tmp["title"], f"({tmp['type']}) @", str(datetime.fromisoformat(tmp["date"]))[:16])
    except:
        print("ERROR: Zeit/Termintyp ungültig")
    print("")
    return dates_sort(dates)

In [10]:
def savejson(data, dates):
    try: #backup
        shutil.copy('archive.json', 'backup/archive'+datetime.now().isoformat()[:19].replace(":","-")+'.json')
    except:
        pass
    #save
    try:
        with open("archive.json", "w", encoding="utf-8") as outfile:
            json.dump(data, outfile, indent = 4)
        with open("dates.json", "w", encoding="utf-8") as outfile:
            json.dump(dates, outfile, indent = 4)
        print("okay. gespeichert")
    except Exception as e:
        print("ERROR beim speichern:", e)

# INTERAKTION HIER
## LADEN

In [11]:
data,dates = loadjson()

okay. Daten geladen


## LISTE BEREINIGEN

In [12]:
data = list_cleanup(data, mal=False)


Eintrag 1:
Chuunibyou demo Koi ga Shitai! Ren
Love, Chunibyo & Other Delusions!: Heart Throb
中二病でも恋がしたい！戀
School
F 12/12



Eintrag 2:
Kokoro Connect
Kokoro Connect
ココロコネクト
School
F 17/13



Eintrag 3:
Fate/Zero
Fate/Zero
フェイト/ゼロ
 
F 13/13



Eintrag 4:
Fate/Zero 2nd Season
Fate/Zero Season 2
フェイト/ゼロ 2ndシーズン
 
F 12/12



Eintrag 5:
Danshi Koukousei no Nichijou
Daily Lives of High School Boys
男子高校生の日常
School
F 12/12



Eintrag 6:
Zankyou no Terror
Terror in Resonance
残響のテロル
Psychological
F 11/11



Eintrag 7:
Tokyo Ghoul
Tokyo Ghoul
東京喰種-トーキョーグール-
Psychological
F 12/12



Eintrag 8:
Kaichou wa Maid-sama!
Maid Sama!
会長はメイド様!
School
F 26/26



Eintrag 9:
Sidonia no Kishi
Knights of Sidonia
シドニアの騎士
Mecha, Space
F 12/12



Eintrag 10:
Shinsekai yori
From the New World
新世界より
Psychological
F 25/25



Eintrag 11:
Girls & Panzer
Girls & Panzer
ガールズ&パンツァー
Military, School
F 12/12



Eintrag 12:
Gekkan Shoujo Nozaki-kun
Monthly Girls' Nozaki-kun
月刊少女野崎くん
School
F 12/12



Eintrag 13:
Fate/stay n

## STATUSUPDATE

In [13]:
data = list_statusupdate(data, flag_needsort=False)

Update-Datum eingeben mit Format: 31.01.2022 >> '3101[2022]' (Punkte und Jahr optional)
Datum: 

STATUS UPDATEN: (Eingabe optional)

Titel: Tsuki to Laika to Nosferatu
Folge: 10/12


Titel: 86 Part 2
Folge: 10/12


Titel: Kimetsu no Yaiba: Yuukaku-hen
Folge: 03/??


Titel: Shiroi Suna no Aquatope
Folge: 24/24


Programm:
0 -- Tsuki to Laika to Nosferatu
1 -- 86 Part 2
2 -- Kimetsu no Yaiba: Yuukaku-hen
3 -- Shiroi Suna no Aquatope
Neue Reihenfolge eingeben mit Format '0123': 

FERTIG


## NEUE SERIE/ALTE REAKTIVIEREN

In [ ]:
data = list_hinzufuegen(data)

## NEUE SITZUNGSTERMINE

In [ ]:
dates = dates_next(dates)

## NEUER EINZELTERMIN

In [ ]:
dates = dates_new(dates)

## SPEICHERN

In [ ]:
savejson(data, dates)